In [54]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

In [2]:
df = pd.read_pickle('./df_3.pkl')
df.head()

,chords,path,melspectogram
0,c,./Guitar_Only/c/c101.wav,"[[-24.261114, -22.469131, -21.94396, -22.46868..."
1,c,./Guitar_Only/c/c7.wav,"[[-41.640045, -32.103657, -31.946598, -38.2757..."
2,c,./Guitar_Only/c/c132.wav,"[[-21.589975, -21.356468, -20.752502, -19.4683..."
3,c,./Guitar_Only/c/c172.wav,"[[-20.76375, -21.655474, -22.655888, -22.58608..."
4,c,./Guitar_Only/c/c158.wav,"[[2.4403708, -3.798201, -22.237926, -22.31823,..."


In [3]:
df.shape

(1757, 3)

In [4]:
chords_mapping = {
    'c':0,
    'dm':1,
    'bm':2,
    'g':3,
    'em':4,
    'a':5,
    'am':6,
    'd':7,
    'f':8,
    'e':9
}

df['chords'] = df['chords'].map(chords_mapping)

In [5]:
df.tail()

,chords,path,melspectogram
1995,9,./Guitar_Only/e/e106.wav,"[[-2.1667492, -6.6485567, -10.442896, -20.4153..."
1996,9,./Guitar_Only/e/e58.wav,"[[-48.91912, -39.279564, -36.134113, -34.0899,..."
1997,9,./Guitar_Only/e/e149.wav,"[[-22.498398, -22.39794, -22.714415, -23.17169..."
1998,9,./Guitar_Only/e/e157.wav,"[[-12.368898, -18.078148, -21.062775, -20.8251..."
1999,9,./Guitar_Only/e/e84.wav,"[[-39.241608, -37.466595, -35.938602, -36.0087..."


In [6]:
df['melspectogram'][0].shape

(128, 65)

In [7]:
df['melspectogram'] = df['melspectogram'].apply(lambda x:x.reshape(1,128,65))

In [8]:
df['melspectogram'][0].shape

(1, 128, 65)

In [9]:
X = df['melspectogram'].to_numpy()
y = df['chords'].to_numpy()

In [10]:
X = np.array([x for x in X], dtype='float32')

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.05, random_state=42, stratify=y)

In [12]:
X_train.shape, X_val.shape

((1669, 1, 128, 65), (88, 1, 128, 65))

In [13]:
X_train_tensor = torch.from_numpy(X_train).type(torch.FloatTensor)
y_train_tensor = torch.from_numpy(y_train).type(torch.LongTensor)

X_val_tensor = torch.from_numpy(X_val).type(torch.FloatTensor)
y_val_tensor = torch.from_numpy(y_val).type(torch.LongTensor)

In [14]:
train = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)

In [15]:
train[3][0].shape

torch.Size([1, 128, 65])

In [16]:
train_loader = torch.utils.data.DataLoader(train, batch_size = 32, shuffle = False)
val_loader = torch.utils.data.DataLoader(val, batch_size = 32, shuffle = False)

In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [18]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1 = nn.Conv2d(1, 32, (5,5), 1)
        self.c2 = nn.Conv2d(32, 32, (5,5), 1)
        self.c3 = nn.Conv2d(32, 64, (3,3), 1)
        self.c4 = nn.Conv2d(64, 64, (3,3), 1)
        self.fc1 = nn.Linear(21504, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.max_pool1 = nn.MaxPool2d((2,2))
        self.max_pool2 = nn.MaxPool2d((2,2), (2,2))
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.relu(self.c1(x))
        x = self.relu(self.c2(x))
        x = self.dropout1(self.max_pool1(x))
        x = self.relu(self.c3(x))
        x = self.relu(self.c4(x))
        x = self.dropout1(self.max_pool2(x))
        x = self.dropout2(self.relu(self.fc1(x.view(x.size(0), -1))))
        x = self.fc2(x)
        
        return x

In [26]:
model = Net()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [27]:
EPOCHS = 20

for epoch in range(EPOCHS):
    running_loss = 0
    
    for i, (img, labels) in enumerate(train_loader):
        img = img.view(-1,1,128,65)
        img = img.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch:{epoch+1}, loss:{running_loss/len(train_loader)}")

    model.eval()
    with torch.no_grad():
        val_acc = 0
        train_acc = 0

        for k, (train_img, train_labels) in enumerate(train_loader):
            train_img = train_img.to(device)
            train_labels = train_labels.to(device)

            train_out = model(train_img)
            train_out = torch.argmax(train_out, axis=1)

            train_acc += torch.sum(train_out == train_labels)/len(train_labels)

        for j, (val_img, val_labels) in enumerate(val_loader):
            val_img = val_img.to(device)
            val_labels = val_labels.to(device)

            val_out = model(val_img)
            val_out = torch.argmax(val_out, axis=1)

            val_acc += torch.sum(val_out == val_labels)/len(val_labels)

        print(f"Train accuracy is {train_acc/(k+1)}")    
        print(f"Validation accuracy is {val_acc/(j+1)}")
        print()
        print("-"*30)
        print()

    model.train()

Epoch:1, loss:1.3748360229831822
Train accuracy is 0.9675707817077637
Validation accuracy is 0.9861111640930176

------------------------------

Epoch:2, loss:0.1394445293501705
Train accuracy is 0.989386796951294
Validation accuracy is 0.9895833730697632

------------------------------

Epoch:3, loss:0.07737222004611537
Train accuracy is 0.990566074848175
Validation accuracy is 1.0

------------------------------

Epoch:4, loss:0.05419028243972516
Train accuracy is 0.9935141801834106
Validation accuracy is 1.0

------------------------------

Epoch:5, loss:0.04854161527842769
Train accuracy is 0.9876179099082947
Validation accuracy is 1.0

------------------------------

Epoch:6, loss:0.04415733326965532
Train accuracy is 0.9970518946647644
Validation accuracy is 1.0

------------------------------

Epoch:7, loss:0.03418118646231102
Train accuracy is 0.9952830076217651
Validation accuracy is 1.0

------------------------------

Epoch:8, loss:0.052676137729971706
Train accuracy is 0.99

In [28]:
torch.save(model.state_dict(),'./model/model_20ep.pt')